In [ ]:
#this script is used for calculating scores of results of Discogan
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive"

Mounted at /content/drive
'biz telecom material.gdoc'
'biz telecom report.gdoc'
 case1.mp4
'Colab Notebooks'
 DiscoGAN
'ECE 9039 Project Proposal.gdoc'
'ECE9063 Project Report.gdoc'
'Eurika UGA solution.gsheet'
'flight meal cost.gdoc'
 horse2zebra
'Migrate for Anthos vs Migrate with Docker Image Deployment.gdoc'
 photo2monet
 pre.mp4
'Silicon Valley Technology Companies Stock Price Prediction.gslides'
'survey_results_public (1).csv.gsheet'
 survey_results_public.csv.gsheet
'telecom assignment 1.gdoc'
 Untitled
'Untitled document.gdoc'


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np



In [ ]:
from math import floor
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
import tensorflow as tf
from PIL import Image
import imageio
import pandas as pd 
import glob
# assumes images have the shape 299x299x3, pixels in [0,255]
def calculate_inception_score(images, n_split=10, eps=1E-16):
	# load inception v3 model
	model = InceptionV3()
	# convert from uint8 to float32
	processed = images.astype('float32')
	# pre-process raw images for inception v3 model
	processed = preprocess_input(processed)
	# predict class probabilities for images
	yhat = model.predict(processed)
	# enumerate splits of images/predictions
	scores = list()
	n_part = floor(images.shape[0] / n_split)
	for i in range(n_split):
		# retrieve p(y|x)
		ix_start, ix_end = i * n_part, i * n_part + n_part
		p_yx = yhat[ix_start:ix_end]
		# calculate p(y)
		p_y = np.expand_dims(np.mean(p_yx, axis=0), 0)
		# calculate KL divergence using log probabilities
		kl_d = p_yx * (np.log(p_yx + eps) - np.log(p_y + eps))
		# sum over classes
		sum_kl_d = kl_d.sum(axis=1)
		# average over images
		avg_kl_d = np.mean(sum_kl_d)
		# undo the log
		is_score = np.exp(avg_kl_d)
		# store
		scores.append(is_score)
	# average across images
	is_avg, is_std = np.mean(scores), np.std(scores)
	return is_avg, is_std

In [ ]:
prediction_list = []
for im_path in glob.glob("/content/drive/My Drive/photo2monet/test_B_out/*.jpg"):
  prediction_list.append(im_path)

In [ ]:
resized_prediction_list = []
for path in (prediction_list):
    prediction = imageio.imread(path)
    prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
    prediction_resized = tf.image.resize(prediction, [299, 299], method=tf.image.ResizeMethod.BILINEAR)
    resized_prediction_list.append(prediction_resized)

resized_prediction_list = np.array(resized_prediction_list)
print(resized_prediction_list.shape)

# calculate inception score
is_avg, is_std = calculate_inception_score(resized_prediction_list)

inception_score = pd.DataFrame(columns=['is_avg', 'is_std'])
inception_score = inception_score.append({'is_avg':is_avg, 'is_std':is_std}, ignore_index=True)
inception_score

(100, 299, 299, 3)


,is_avg,is_std
0,1.894119,0.238353


In [ ]:
from scipy.linalg import sqrtm
 
# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = np.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if np.iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

In [ ]:
monet_list = []
for im_path in glob.glob("/content/drive/My Drive/photo2monet/testA/*.jpg"):
  monet = imageio.imread(im_path)
  monet = (monet * 127.5 + 127.5).astype(np.uint8)
  monet_list.append(monet)
monet_list = np.array(monet_list)
print(monet_list.shape)

(33, 256, 256, 3)


In [ ]:
# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

# convert integer to floating point values
monet_list = monet_list.astype('float32')

# resize images
resized_monet_list = []
for img in (monet_list):

    img_resized = tf.image.resize(img, [299, 299], method=tf.image.ResizeMethod.BILINEAR)
    resized_monet_list.append(img_resized)

resized_monet_list = np.array(resized_monet_list)
print(resized_monet_list.shape)

# pre-process images
images1 = preprocess_input(resized_monet_list)
images2 = preprocess_input(resized_prediction_list)

# fid between images1 and images1
fid_same = calculate_fid(model, images1, images1)
# fid between images1 and images2
fid_diff = calculate_fid(model, images1, images2)

fid_score = pd.DataFrame(columns=['FID_same', 'FID_diff'])
fid_score = fid_score.append({'FID_same': fid_same, 'FID_diff':fid_diff}, ignore_index=True)
fid_score

(33, 299, 299, 3)


,FID_same,FID_diff
0,-0.000111,213.792491
